## Choosing Model
For the frontier model, I am using OpenAi's O1-mini with an ELO rating of 1237.00±31.00 (https://scale.com/leaderboard/coding)

In [ ]:
%pip install anthropic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.2/297.2 kB 20.9 MB/s eta 0:00:00


In [ ]:
## imports
import os
import requests
from google.colab import userdata
from openai import OpenAI
import google.generativeai
from anthropic import Anthropic
import gradio as gr
import subprocess
from IPython.display import Markdown, display, update_display
from huggingface_hub import login

In [ ]:
HF_TOKEN = userdata.get('HF_TOKEN')
openai_api = userdata.get('OPENAI_API_KEY')
gemini_api = userdata.get('GOOGLE_API_KEY')
claude_api = userdata.get('ANTHROPIC_API_KEY')

In [ ]:
## login to hf
login(HF_TOKEN)

openai_client = OpenAI(api_key=openai_api)

gemini_client = OpenAI(
    api_key=gemini_api,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

claude_client = Anthropic(api_key=claude_api)

In [ ]:
OPENAI_MODEL = 'gpt-4o'
CLAUDE_MODEL = 'claude-sonnet-4-0'
GEMINI_MODEL = 'gemini-2.5-pro'
OPENSOURCE_MODEL = 'Qwen/CodeQwen1.5-7B-Chat'

In [ ]:
## System prompt
system_prompt ="You are a very helpful assistant that can reimplement python codes into high performing C++ for google colab"
system_prompt+="You are to provide only C++ code. use comments appropriately and do not provide any explanation other than comments."
system_prompt+="The C++ version needs to reproduce the exact same output that the python code does. the execution time for the C++ should be the fastest"

In [ ]:
def get_user_prompt(python_code):
  user_prompt="rewrite the python code in C++ with the fastest possible execution. the C++ should have identical outcomes like the python in the least time."
  user_prompt+="respond only in C++ code. dont explain any bit other than the comments."
  user_prompt+="pay attention to the data types. Make sure there is no int overflow. Also make sure to include all the necessary libraries for C++ like iomanip."
  user_prompt+=f"The following one is the python code that you need to convert into C++: {python_code}"
  return user_prompt

In [ ]:
def get_messages(python_code):
  return [
      {"role": "system", "content": system_prompt},
      {"role": "user", "content": get_user_prompt(python_code)}
  ]

In [ ]:
## create a new file called "optimized.cpp"
def write_output(cpp):
  code = cpp.replace("```cpp", "").replace("```", "")
  with open("optimized.cpp", "w") as f:
    f.write(code)

In [ ]:
def optimize_gpt(python_code):
  stream = openai_client.chat.completions.create(
      model = OPENAI_MODEL,
      messages = get_messages(python_code),
      stream = True
  )
  reply =""
  for chunk in stream:
    piece = chunk.choices[0].delta.content or ""
    reply+=piece
    print(piece, end="", flush=True)
  write_output(reply)

In [ ]:
def optimize_claude(python_code):
  result = claude_client.messages.stream(
      model = CLAUDE_MODEL,
      max_tokens=2000,
      system=system_prompt,
      messages=[{"role": "user", "content": get_user_prompt(python_code)}]
  )
  reply =""
  with result as stream:
    for text in stream.text_stream:
      reply+=text
      print(text, end="", flush=True)
  write_output(reply)

In [ ]:
def optimize_gemini(python_code):
  stream = gemini_client.chat.completions.create(
      model = GEMINI_MODEL,
      messages = get_messages(python_code),
      stream = True
  )
  reply =""
  for chunk in stream:
    piece = chunk.choices[0].delta.content or ""
    reply+=piece
    print(piece, end="", flush=True)
  write_output(reply)

In [ ]:
!pip install -q --upgrade torch==2.5.1+cu124 torchvision==0.20.1+cu124 torchaudio==2.5.1+cu124 --index-url https://download.pytorch.org/whl/cu124
!pip install -q requests bitsandbytes==0.46.0 transformers==4.48.3 accelerate==1.3.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.2/908.2 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 136.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 105.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 101.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 128.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 37.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig
import torch

In [ ]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(OPENSOURCE_MODEL, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    OPENSOURCE_MODEL,
    quantization_config=quantization_config,
    device_map=None,
    trust_remote_code=True
).to("cuda")
model.eval()

tokenizer_config.json:   0%|          | 0.00/972 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/1.42M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


...

model-00003-of-00004.safetensors:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/2.71G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(92416, 4096)
    (layers): ModuleList(
      (0-31): 32 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=True)
          (k_proj): Linear4bit(in_features=4096, out_features=512, bias=True)
          (v_proj): Linear4bit(in_features=4096, out_features=512, bias=True)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=13440, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=13440, bias=False)
          (down_proj): Linear4bit(in_features=13440, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): Qwen2RMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): Qwen2RMSNorm((4096,), e

In [ ]:
from transformers import TextIteratorStreamer
import threading

def optimize_qwen(python_code):
    ## build prompt
    inputs = tokenizer.apply_chat_template(
        get_messages(python_code),
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(getattr(model, "device", "cuda"))

    if tokenizer.pad_token_id is None and tokenizer.eos_token_id is not None:
        tokenizer.pad_token = tokenizer.eos_token

    # streamer that yields text chunks
    streamer = TextIteratorStreamer(
        tokenizer,
        skip_prompt=True,
        skip_special_tokens=True
    )

    gen_kwargs = dict(
        inputs=inputs,
        max_new_tokens=2000,
        do_sample=True,
        temperature=0.7,
        top_p=0.95,
        streamer=streamer
    )

    # run generation in a background thread so we can iterate chunks
    thread = threading.Thread(target=model.generate, kwargs=gen_kwargs)
    thread.start()

    reply_parts = []
    for piece in streamer:
        reply_parts.append(piece or "")
        print(piece or "", end="", flush=True)

    reply = "".join(reply_parts).strip()
    write_output(reply)
    return reply # Return the generated C++ code

In [ ]:
pi = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(100_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [ ]:
exec(pi)

Result: 3.141592658589
Execution Time: 16.645491 seconds


In [ ]:
optimize_gpt(pi)

```cpp
#include <iostream>
#include <iomanip>
#include <chrono>

// Function to perform the calculation
double calculate(int iterations, double param1, double param2) {
    double result = 1.0; // Start with the initial result of 1.0
    for (int i = 1; i <= iterations; ++i) {
        // Calculate j for subtraction
        double j = i * param1 - param2;
        result -= (1.0 / j);
        
        // Calculate j for addition
        j = i * param1 + param2;
        result += (1.0 / j);
    }
    return result;
}

int main() {
    // Start the time measurement
    auto start_time = std::chrono::high_resolution_clock::now();
    
    // Perform the calculation
    double result = calculate(100000000, 4.0, 1.0) * 4;
    
    // End the time measurement
    auto end_time = std::chrono::high_resolution_clock::now();
    
    // Calculate the execution time in seconds
    std::chrono::duration<double> elapsed_seconds = end_time - start_time;
    
    // Output the results with formatting
 

In [ ]:
!g++ optimized.cpp -o optimized && ./optimized

Result: 3.141592658589
Execution Time: 0.635053 seconds


In [ ]:
optimize_claude(pi)

```cpp
#include <iostream>
#include <iomanip>
#include <chrono>

double calculate(int iterations, int param1, int param2) {
    double result = 1.0;
    for (int i = 1; i <= iterations; ++i) {
        int j1 = i * param1 - param2;
        int j2 = i * param1 + param2;
        result -= (1.0 / j1);
        result += (1.0 / j2);
    }
    return result;
}

int main() {
    auto start_time = std::chrono::high_resolution_clock::now();
    double result = calculate(100000000, 4, 1) * 4;
    auto end_time = std::chrono::high_resolution_clock::now();
    
    auto duration = std::chrono::duration_cast<std::chrono::microseconds>(end_time - start_time);
    double execution_time = duration.count() / 1000000.0;
    
    std::cout << std::fixed << std::setprecision(12);
    std::cout << "Result: " << result << std::endl;
    std::cout << std::setprecision(6);
    std::cout << "Execution Time: " << execution_time << " seconds" << std::endl;
    
    return 0;
}
```

In [ ]:
!g++ optimized.cpp -o optimized && ./optimized

Result: 3.141592658589
Execution Time: 0.636850 seconds


In [ ]:
optimize_gemini(pi)

```cpp
// To compile this code in Google Colab for maximum performance,
// save it as a file (e.g., solution.cpp) and run the following command in a cell:
// !g++ -O3 -fopenmp -o solution solution.cpp && ./solution

#include <iostream>      // For input/output operations (std::cout)
#include <iomanip>       // For output formatting (std::fixed, std::setprecision)
#include <chrono>        // For high-resolution timing
#include <omp.h>         // For OpenMP for parallel processing

/**
 * @brief Calculates a series approximation, optimized and parallelized.
 *
 * This function calculates the sum: 1.0 + sum_{i=1 to N} [ 1/(p1*i + p2) - 1/(p1*i - p2) ]
 * The formula inside the sum is algebraically simplified to -2*p2 / ((p1*i)^2 - p2^2)
 * to reduce the number of operations, especially expensive divisions, per iteration.
 * The loop is parallelized using OpenMP for significant speedup on multi-core CPUs.
 *
 * @param iterations The number of iterations (N) for the series.
 * @param param1

In [ ]:
!g++ optimized.cpp -o optimized && ./optimized

Result: 3.141592658097
Execution Time: 0.315644 seconds


In [ ]:
optimize_qwen(pi)

Here is the equivalent C++ code for the Python code provided. This version of the code uses the standard C++ libraries and runs in about 7 seconds on my machine:

```cpp
#include <iostream>
#include <iomanip>
#include <chrono>

double calculate(int iterations, double param1, double param2) {
    double result = 1.0;
    for(int i = 1; i <= iterations; ++i) {
        double j = i * param1 - param2;
        result -= 1 / j;
        j = i * param1 + param2;
        result += 1 / j;
    }
    return result;
}

int main() {
    auto start_time = std::chrono::high_resolution_clock::now();

    double result = calculate(100000000, 4, 1) * 4;

    auto end_time = std::chrono::high_resolution_clock::now();

    std::cout << "Result: " << std::setprecision(12) << result << std::endl;
    std::cout << "Execution Time: " << std::chrono::duration_cast<std::chrono::microseconds>(end_time - start_time).count() / 1000.0 << " milliseconds" << std::endl;

    return 0;
}
```

This C++ code does exactly 

In [ ]:
!g++ optimized.cpp -o optimized && ./optimized

Result: 3.14159265859
Execution Time: 633.349 milliseconds


In [ ]:
python_hard = """# Be careful to support large number sizes

def lcg(seed, a=1664525, c=1013904223, m=2**32):
    value = seed
    while True:
        value = (a * value + c) % m
        yield value

def max_subarray_sum(n, seed, min_val, max_val):
    lcg_gen = lcg(seed)
    random_numbers = [next(lcg_gen) % (max_val - min_val + 1) + min_val for _ in range(n)]
    max_sum = float('-inf')
    for i in range(n):
        current_sum = 0
        for j in range(i, n):
            current_sum += random_numbers[j]
            if current_sum > max_sum:
                max_sum = current_sum
    return max_sum

def total_max_subarray_sum(n, initial_seed, min_val, max_val):
    total_sum = 0
    lcg_gen = lcg(initial_seed)
    for _ in range(20):
        seed = next(lcg_gen)
        total_sum += max_subarray_sum(n, seed, min_val, max_val)
    return total_sum

# Parameters
n = 10000         # Number of random numbers
initial_seed = 42 # Initial seed for the LCG
min_val = -10     # Minimum value of random numbers
max_val = 10      # Maximum value of random numbers

# Timing the function
import time
start_time = time.time()
result = total_max_subarray_sum(n, initial_seed, min_val, max_val)
end_time = time.time()

print("Total Maximum Subarray Sum (20 runs):", result)
print("Execution Time: {:.6f} seconds".format(end_time - start_time))
"""

In [ ]:
exec(python_hard)

Total Maximum Subarray Sum (20 runs): 10980
Execution Time: 61.547552 seconds


In [ ]:
optimize_gpt(python_hard)

```cpp
#include <iostream>
#include <vector>
#include <limits>
#include <chrono>

using namespace std;

// Linear Congruential Generator function
unsigned long lcg(unsigned long seed, unsigned long a = 1664525, unsigned long c = 1013904223, unsigned long m = 4294967296) {
    return (a * seed + c) % m;
}

// Max Subarray Sum function
long long max_subarray_sum(int n, unsigned long seed, int min_val, int max_val) {
    vector<int> random_numbers(n);
    
    // Generate random numbers using LCG
    for (int i = 0; i < n; ++i) {
        seed = lcg(seed);
        random_numbers[i] = seed % (max_val - min_val + 1) + min_val;
    }
    
    long long max_sum = numeric_limits<long long>::min();
    
    // Calculate max subarray sum
    for (int i = 0; i < n; ++i) {
        long long current_sum = 0;
        for (int j = i; j < n; ++j) {
            current_sum += random_numbers[j];
            if (current_sum > max_sum) {
                max_sum = current_sum;
            }
        }
    }


In [ ]:
!g++ optimized.cpp -o optimized && ./optimized

Total Maximum Subarray Sum (20 runs): 10980
Execution Time: 3.61865 seconds


In [ ]:
optimize_claude(python_hard)

```cpp
#include <iostream>
#include <vector>
#include <algorithm>
#include <climits>
#include <chrono>
#include <iomanip>

class LCG {
private:
    uint64_t value;
    static const uint64_t a = 1664525ULL;
    static const uint64_t c = 1013904223ULL;
    static const uint64_t m = 4294967296ULL; // 2^32

public:
    LCG(uint64_t seed) : value(seed) {}
    
    uint64_t next() {
        value = (a * value + c) % m;
        return value;
    }
};

long long max_subarray_sum(int n, uint64_t seed, int min_val, int max_val) {
    LCG lcg_gen(seed);
    std::vector<int> random_numbers;
    random_numbers.reserve(n);
    
    // Generate random numbers
    int range = max_val - min_val + 1;
    for (int i = 0; i < n; ++i) {
        random_numbers.push_back(static_cast<int>(lcg_gen.next() % range) + min_val);
    }
    
    // Kadane's algorithm for maximum subarray sum
    long long max_sum = LLONG_MIN;
    long long current_sum = 0;
    
    for (int i = 0; i < n; ++i) {
        current_sum =

In [ ]:
!g++ optimized.cpp -o optimized && ./optimized

Total Maximum Subarray Sum (20 runs): 10980
Execution Time: 0.013289 seconds


In [ ]:
optimize_gemini(python_hard)

```cpp
#include <iostream>      // For input/output streams (e.g., std::cout)
#include <vector>        // For std::vector, though not used in the final optimized version
#include <chrono>        // For high-precision timing
#include <cstdint>       // For fixed-width integer types like uint32_t, uint64_t
#include <limits>        // For std::numeric_limits to get the minimum value for a type
#include <iomanip>       // For output formatting (e.g., std::fixed, std::setprecision)

// LCG (Linear Congruential Generator) constants, matching the Python code.
// 'm' is 2^32, which is handled implicitly by using the 32-bit unsigned integer type 'uint32_t'.
// Unsigned integer arithmetic naturally wraps around, performing the modulo operation automatically.
const uint32_t a = 1664525;
const uint32_t c = 1013904223;

// LCG function to generate the next pseudo-random number.
// The seed is passed by reference to maintain state across calls.
// The intermediate product (a * seed) is cast to uint6

In [ ]:
!g++ optimized.cpp -o optimized && ./optimized

Total Maximum Subarray Sum (20 runs): 10980
Execution Time: 0.002955 seconds


In [ ]:
optimize_qwen(python_hard)

#include <iostream>
#include <vector>
#include <random>
#include <algorithm>
#include <iomanip>

// LCG generator function
std::mt19937 lcg(42);

// Function to generate random numbers
std::vector<int> generate_random_numbers(int n, int min_val, int max_val) {
    std::uniform_int_distribution<> dis(min_val, max_val);
    std::vector<int> random_numbers(n);
    for (int i = 0; i < n; i++) {
        random_numbers[i] = dis(lcg);
    }
    return random_numbers;
}

// Function to find maximum subarray sum
int max_subarray_sum(std::vector<int>& random_numbers) {
    int max_sum = std::numeric_limits<int>::min();
    for (int i = 0; i < random_numbers.size(); i++) {
        int current_sum = 0;
        for (int j = i; j < random_numbers.size(); j++) {
            current_sum += random_numbers[j];
            max_sum = std::max(max_sum, current_sum);
        }
    }
    return max_sum;
}

// Function to calculate total maximum subarray sum
int total_max_subarray_sum(int n, int min_val, int 

In [ ]:
!g++ optimized.cpp -o optimized && ./optimized

Total Maximum Subarray Sum (20 runs): 14395
Execution Time: 7.212873 seconds


In [ ]:
def stream_gpt(python_code):
  stream = openai_client.chat.completions.create(
      model = OPENAI_MODEL,
      messages = get_messages(python_code),
      stream = True
  )
  reply =""
  for chunk in stream:
    piece = chunk.choices[0].delta.content or ""
    reply+=piece
    yield reply.replace("```cpp\n","").replace("```", "")


def stream_claude(python_code):
  reply = claude_client.messages.stream(
      model = CLAUDE_MODEL,
      max_tokens=2000,
      system=system_prompt,
      messages=[{"role": "user", "content": get_user_prompt(python_code)}]
  )
  response =""
  with reply as stream:
    for text in stream.text_stream:
      response+=text
      yield response.replace("```cpp\n","").replace("```", "")

def stream_gemini(python_code):
  stream = openai_client.chat.completions.create(
      model = GEMINI_MODEL,
      messages = get_messages(python_code),
      stream = True
  )
  reply =""
  for chunk in stream:
    piece = chunk.choices[0].delta.content or ""
    reply+=piece
    yield reply.replace("```cpp\n","").replace("```", "")

def stream_qwen(python_code):
  ## build prompt
    inputs = tokenizer.apply_chat_template(
        get_messages(python_code),
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(getattr(model, "device", "cuda"))

    if tokenizer.pad_token_id is None and tokenizer.eos_token_id is not None:
        tokenizer.pad_token = tokenizer.eos_token

    # streamer that yields text chunks
    streamer = TextIteratorStreamer(
        tokenizer,
        skip_prompt=True,
        skip_special_tokens=True
    )

    gen_kwargs = dict(
        inputs=inputs,
        max_new_tokens=2000,
        do_sample=True,
        temperature=0.7,
        top_p=0.95,
        streamer=streamer
    )

    # run generation in a background thread so we can iterate chunks
    thread = threading.Thread(target=model.generate, kwargs=gen_kwargs)
    thread.start()

    reply = ""
    for piece in streamer:
        piece = piece or ""
        reply += piece
        # keep same cleanup as others
        yield reply.replace("```cpp\n", "").replace("```", "")

In [ ]:
def optimize(python_code, model_dropdown):
  model = model_dropdown # Access the value of the dropdown
  if model=="GPT":
    result = stream_gpt(python_code)
  elif model=="CLAUDE":
    result = stream_claude(python_code)
  elif model=="GEMINI":
    result = stream_gemini(python_code)
  elif model=="QWEN":
    result = stream_qwen(python_code)
  else:
    raise ValueError("Unknown Model")
  for strem_so_far in result:
    yield strem_so_far

In [ ]:
def execute_python(python_code):
  try:
    result = subprocess.run(['python', '-c', python_code], capture_output=True, text=True, check=True)
    return result.stdout
  except subprocess.CalledProcessError as e:
    return f"Error:\n{e.stderr}"

In [ ]:
def execute_cpp(cpp_code):
  try:
    with open("temp.cpp", "w") as f:
      f.write(cpp_code)

    compile_result = subprocess.run(['g++', "temp.cpp", '-o', 'temp_cpp_executable'], capture_output=True, text=True, check=True)
    if compile_result.stderr:
      return f"Compilation Errors:\n{compile_result.stderr}"

    run_result = subprocess.run(['./temp_cpp_executable'], capture_output=True, text=True, check=True)
    return run_result.stdout
  except subprocess.CalledProcessError as e:
    return f"Execution Error:\n{e.stderr}"
  except FileNotFoundError:
    return "Error: g++ compiler not found. Please ensure it is installed."

In [ ]:
with gr.Blocks() as ui:
  with gr.Row():
    python=gr.Textbox(label="Python Code: ", lines = 15, value=pi)
    cpp = gr.Textbox(label="C++ Code: ", lines=15)
  with gr.Row():
    model = gr.Dropdown(["GPT", "CLAUDE", "GEMINI", "QWEN"], label="Select Model", value="GPT")
    convert = gr.Button("Convert Code")

  convert.click(optimize, inputs = [python, model], outputs = [cpp])

ui.launch(debug=True, inbrowser=True)

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://c8deea8fa9409e1af1.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


KeyboardInterrupt: 

In [ ]:
from time import perf_counter
import plotly.graph_objects as go


def build_time_plotly(times_dict):
    labels, values = [], []
    if times_dict.get("python") is not None:
        labels.append("Python"); values.append(times_dict["python"])
    if times_dict.get("cpp") is not None:
        labels.append("C++"); values.append(times_dict["cpp"])

    # always build both bars if one exists, so the axis doesn’t jump
    if not labels:
        labels, values = ["Python","C++"], [0,0]

    fig = go.Figure(
        data=[go.Bar(x=labels, y=values, text=[f"{v:.4f}s" for v in values],
                     textposition="outside")]
    )
    fig.update_layout(
        title="Execution Time Comparison",
        yaxis_title="Seconds",
        xaxis_tickangle=0,
        margin=dict(l=40, r=20, t=50, b=40),
        height=360
    )
    return fig

def run_python_and_time(python_code, times):
    t0 = perf_counter()
    out = execute_python(python_code)
    dt = perf_counter() - t0
    times = dict(times or {})
    times["python"] = dt
    fig = build_time_plotly(times)
    summary = []
    if times.get("python") is not None: summary.append(f"**Python:** {times['python']:.6f} s")
    if times.get("cpp")    is not None: summary.append(f"**C++:** {times['cpp']:.6f} s")
    return out, times, fig, " • ".join(summary)

def run_cpp_and_time(cpp_code, times):
    t0 = perf_counter()
    out = execute_cpp(cpp_code)
    dt = perf_counter() - t0
    times = dict(times or {})
    times["cpp"] = dt
    fig = build_time_plotly(times)
    summary = []
    if times.get("python") is not None: summary.append(f"**Python:** {times['python']:.6f} s")
    if times.get("cpp")    is not None: summary.append(f"**C++:** {times['cpp']:.6f} s")
    return out, times, fig, " • ".join(summary)

with gr.Blocks() as ui:
    # start with an empty dict so we don’t accidentally keep Nones
    times_state = gr.State({})

    with gr.Row():
        python = gr.Textbox(label="Python Code:", lines=15, value=pi)
        cpp    = gr.Textbox(label="C++ Code:", lines=15)

    with gr.Row():
        model = gr.Dropdown(["GPT", "CLAUDE", "GEMINI", "QWEN"], label="Select Model", value="GPT")
        convert = gr.Button("Convert Code")

    with gr.Row():
        run_py  = gr.Button("Run Python ▶")
        run_cpp = gr.Button("Run C++ ▶")

    with gr.Row():
        py_out  = gr.Textbox(label="Python Output", lines=10)
        cpp_out = gr.Textbox(label="C++ Output",  lines=10)

    with gr.Row():
        time_plot = gr.Plot(label="Execution Time Comparison")  # Plotly figure will render here
        time_text = gr.Markdown()

    convert.click(fn=optimize, inputs=[python, model], outputs=[cpp])
    run_py.click(fn=run_python_and_time, inputs=[python, times_state],
                 outputs=[py_out, times_state, time_plot, time_text])
    run_cpp.click(fn=run_cpp_and_time, inputs=[cpp, times_state],
                  outputs=[cpp_out, times_state, time_plot, time_text])

ui.launch(debug=True, inbrowser=True)